# NTDS'18 tutorial 2: build a graph from features
[Benjamin Ricaud](https://people.epfl.ch/benjamin.ricaud), [EPFL LTS2](https://lts2.epfl.ch)

* Dataset: [Iris](https://archive.ics.uci.edu/ml/datasets/Iris)
* Tools: [pandas](https://pandas.pydata.org), [numpy](http://www.numpy.org), [scipy](https://www.scipy.org), [matplotlib](https://matplotlib.org), [networkx](https://networkx.github.io), [gephi](https://gephi.org/)

## Tools

The below line is a [magic command](https://ipython.readthedocs.io/en/stable/interactive/magics.html) that allows plots to appear in the notebook.

In [ ]:
%matplotlib inline

The first thing is always to import the packages we'll use.

In [ ]:
import pandas as pd
import numpy as np
from scipy.spatial.distance import pdist, squareform
from matplotlib import pyplot as plt
import networkx as nx

Tutorials on pandas can be found at:
* <https://pandas.pydata.org/pandas-docs/stable/10min.html>
* <https://pandas.pydata.org/pandas-docs/stable/tutorials.html>

Tutorials on numpy can be found at:
* <https://docs.scipy.org/doc/numpy/user/quickstart.html>
* <http://www.scipy-lectures.org/intro/numpy/index.html>
* <http://www.scipy-lectures.org/advanced/advanced_numpy/index.html>

A tutorial on networkx can be found at:
* <https://networkx.github.io/documentation/stable/tutorial.html>

## Import and explore the data

We will play with famous Iris dataset. This dataset can be found in many places on the net and was first released at <https://archive.ics.uci.edu/ml/index.php>. For example it is stored on [Kaggle](https://www.kaggle.com/uciml/iris/), with many demos and Jupyter notebooks you can test (have a look at the "kernels" tab).

![Iris Par Za — Travail personnel, CC BY-SA 3.0, https://commons.wikimedia.org/w/index.php?curid=144395](figures/iris_germanica.jpg)

In [ ]:
iris = pd.read_csv('data/iris.csv')
iris.head()

The description of the entries is given here:
https://www.kaggle.com/uciml/iris/home

In [ ]:
iris['Species'].unique()

In [ ]:
iris.describe()

## Build a graph from the features

We are going to build a graph from this data. The idea is to represent iris samples (rows of the table) as nodes, with connections depending on their physical similarity.

The main question is how to define the notion of similarity between the flowers. For that, we need to introduce a measure of similarity. It should use the properties of the flowers and provide a positive real value for each pair of samples.

Let us separate the data into two parts: physical properties and labels.

In [ ]:
features = iris.loc[:, ['SepalLengthCm', 'SepalWidthCm', 'PetalLengthCm', 'PetalWidthCm']]
species = iris.loc[:, 'Species']

In [ ]:
features.head()

In [ ]:
species.head()

The `pdist` function computes the pairwise distance. By default it is the Euclidian distance. `features.values` is a numpy array extracted from the Pandas dataframe. Very handy.

In [ ]:
pdist?

In [ ]:
distances = pdist(features.values)

In [ ]:
# Turn the list of weights into a matrix.
adjacency = squareform(distances)

Sometimes, you may need to compute additional features before processing them with some machine learning or some other data processing step. With Pandas, it is as simple as that:

In [ ]:
# Compute a new column using the existing ones.
features['SepalLengthSquared'] = features['SepalLengthCm']**2
features.head()

Coming back to the weight matrix, we have obtained a full matrix but we may not need all the connections (reducing the number of connections saves some space and computations!). We can sparsify the graph by removing the values (edges) below some fixed threshold. Let us see what kind of threshold we could use:

In [ ]:
plt.hist(distances)
plt.title('Distribution of distances')
plt.show()

In [ ]:
# Let us choose a threshold of 3.
adjacency[adjacency < 3] = 0

## Graph visualization

To conclude, let us visualize the graph. We will use the python module networkx.

In [ ]:
# A simple command to create the graph from the adjacency matrix.
graph = nx.from_numpy_array(adjacency)

Let us try some direct visualizations using networkx.

In [ ]:
nx.draw_spectral(graph)

Oh! It seems to be separated in 3 parts! Are they related to the 3 different species of iris?

Let us try another [layout algorithm](https://en.wikipedia.org/wiki/Graph_drawing#Layout_methods), where the edges are modeled as springs.

In [ ]:
nx.draw_spring(graph)

Save the graph to disk in the `gexf` format, readable by gephi and other tools that manipulate graphs. You may now explore the graph using gephi and compare the visualizations.

In [ ]:
nx.write_gexf(graph,'iris.gexf')